# Analyze DTI

In [1]:
# import packages
# -----------------------------------------------------------------------------
# import pip
# pip.main(['install', 'seaborn']) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import sys
import json

In [2]:
# parameters
# -----------------------------------------------------------------------------
data_path = '/Users/ranigera/Dropbox/DTI_tests'
#data_path = '/Volumes/backup_raw_data/Rani_HIS_DATA_HD/BIDS'
preproc_path = '/Users/ranigera/Dropbox/DTI_tests/preproc'

expectedVolums = {
    'AP': 69,
    'PA' : 7,
    }
expectedB0s_indxs = {
    'AP_before': [0, 1, 18, 35, 52],
    'PA_before': [0, 2, 3, 4, 5, 6],
    'AP_after': [0, 1, 18, 35, 52],
    'PA_after': [0, 2, 3, 4, 5, 6]
    }


In [3]:
# Get folders and remove excluded subjects
# -----------------------------------------------------------------------------
print('>> Get sub folders')
subjFolders = [el for el in os.listdir(data_path) if 'sub' in el]

print('>> Get exclusion list')
with open('/Users/ranigera/Google_Drive_TAU/Experiments/HIS_STUDY/Analysis/codes/paths_and_vars.py') as txtFile:
    txt = txtFile.read()
participantsToExclude = [int(el) for el in txt.split('participantsToExclude = [')[1].split(']')[0].replace('\n','').replace('\n','').replace("'","").split(',')]

print('>> Remove sub folders of excluded participants in case they are there')
subjFolders = [el for el in subjFolders if int(el.split('-')[1]) not in participantsToExclude]


>> Get sub folders
>> Get exclusion list
>> Remove sub folders of excluded participants in case they are there


## Check for missing scans or wrong phase encoding directions for ALL SUBJECTS 

In [4]:
print ('>> Verify that all the scans exist and that the phase encoding directions are as they should.')
subjectsWithAProblem = []
for subjFolder in subjFolders:
    sub = int(subjFolder.split("-",1)[1])
    group = '1day' if sub < 200 else '3day'
    last_sess = group[0]
    DWI_path_before = os.path.join(data_path, subjFolder, 'ses-1/dwi/')
    DWI_path_after = os.path.join(data_path, subjFolder, f'ses-{last_sess}/dwi/')
    scansBaseNames = {
        'AP_before': f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-ap_run-01_dwi")}',
        'PA_before' : f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-pa_run-01_dwi")}',
        'AP_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-ap_run-02_dwi")}',
        'PA_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-pa_run-02_dwi")}'
        }

    for scan in scansBaseNames.keys():
        # print(scansBaseNames[scan] + '.json')
        # print(scanData['PhaseEncodingDirection'])
        if not os.path.exists(scansBaseNames[scan] + '.json'):
            subjectsWithAProblem.append(sub)
            print(' *** Scan not found: ' + scansBaseNames[scan] + '.json')
            continue
        with open(scansBaseNames[scan] + '.json') as json_file:        
            scanData = json.load(json_file)
            if ('acq-ap_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j-') or \
                ('acq-pa_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j'):
                subjectsWithAProblem.append(sub)
                print(' *** There is a problem with the scanning directions: ' + scansBaseNames[scan] + '.json is defined as ' + scanData['PhaseEncodingDirection'] + '.')
                continue

subjectsWithAProblem = list(set(subjectsWithAProblem))
subjectsWithAProblem.sort()

>> Verify that all the scans exist and that the phase encoding directions are as they should.
 *** Scan not found: /Users/ranigera/Dropbox/DTI_tests/sub-211/ses-3/dwi/sub-211_ses-3_acq-ap_run-02_dwi.json
 *** Scan not found: /Users/ranigera/Dropbox/DTI_tests/sub-211/ses-3/dwi/sub-211_ses-3_acq-pa_run-02_dwi.json
 *** There is a problem with the scanning directions: /Users/ranigera/Dropbox/DTI_tests/sub-110/ses-1/dwi/sub-110_ses-1_acq-pa_run-01_dwi.json is defined as j-.
 *** There is a problem with the scanning directions: /Users/ranigera/Dropbox/DTI_tests/sub-110/ses-1/dwi/sub-110_ses-1_acq-pa_run-02_dwi.json is defined as j-.


# [TEMP] Working on on subject (for now)

In [5]:
scansBaseNames.keys()

dict_keys(['AP_before', 'PA_before', 'AP_after', 'PA_after'])

## Get b0 volume indices and perform bval & bvec QA

In [6]:

print('> Verify that data points in the bval files is as expected.')
if pd.read_csv(scansBaseNames['AP_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['PA'] or \
    pd.read_csv(scansBaseNames['AP_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['PA']:
        print(f' *** The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')
        raise Exception(f'The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')

print('> Verify that data points in the bvec files is as expected.')
if pd.read_csv(scansBaseNames['AP_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['PA'] or \
    pd.read_csv(scansBaseNames['AP_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['PA']:
        print(f' *** The number of data points in the bvec for one of the scans for subjetc {sub} is not as expected.')
        raise ValueError(f'The number of data points in the bvec for one of the scans for subjetc ' + str(sub) + ' is not as expected.')


print('> Extract B0s:')
B0s_indxs = {}
for scan in scansBaseNames.keys():
    B0s=pd.read_csv(scansBaseNames[scan] + '.bval', header=None, sep=' ').T
    B0s.columns = ['bval']
    B0s_indxs[scan] = list(B0s[B0s.bval < 20].index)

print('> Verify that b0 quantity and indices are as expected.')
if B0s_indxs != expectedB0s_indxs:
    print(f' *** The indices of the b0s for one of the scans for subjetc {sub} are not as expected.')
    raise ValueError(f'The indices of the b0s for one of the scans for subjetc ' + str(sub) + ' are not as expected.')


> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.


## Preprocessing

### (1) TOPUP
* A tool for estimating and correcting susceptibility induced distortions

#### Create pre-processing folder

In [7]:
print('> Create preprocessing folder')
try:
    os.makedirs(os.path.join(preproc_path, subjFolder), exist_ok=False)
except:
    pass

> Create preprocessing folder


#### Extract B0 volumes

In [8]:
print('> Extract B0s (using the fslroi):')
sub_B0s_files = []
for scan in scansBaseNames.keys():
    for B0ind in B0s_indxs[scan]:
        B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
        print(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
        os.system(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
        sub_B0s_files.append(B0_file_name)


> Extract B0s (using the fslroi):
fslroi /Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-0.nii.gz 0 1
fslroi /Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-1.nii.gz 1 1
fslroi /Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-18.nii.gz 18 1
fslroi /Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-35.nii.gz 35 1
fslroi /Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-52.nii.gz 52 1
fslroi /Users/r

#### Merge B0 volumes for TOPUP

In [90]:
print('> Merge B0s for each session (using the fslmerge):')
#print([scan for scan in sub_B0s_files if 'AP_before' in scan])
#print([scan for scan in sub_B0s_files if 'PA_before' in scan])
print(f'fslmerge -t {os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_before_b0s' + ' ' + ' '.join([scan for scan in sub_B0s_files if 'AP_before' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if 'PA_before' in scan]))
os.system(f'fslmerge -t {os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_before_b0s' + ' ' + ' '.join([scan for scan in sub_B0s_files if 'AP_before' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if 'PA_before' in scan]))
print(f'fslmerge -t {os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_after_b0s' + ' ' + ' '.join([scan for scan in sub_B0s_files if 'AP_after' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if 'PA_after' in scan]))
os.system(f'fslmerge -t {os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_after_b0s' + ' ' + ' '.join([scan for scan in sub_B0s_files if 'AP_after' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if 'PA_after' in scan]))


> Merge B0s for each session (using the fslmerge):
fslmerge -t /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_PA_before_b0s /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-0.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-1.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-18.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-35.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_before_b0_volInd-52.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_PA_before_b0_volInd-0.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_PA_before_b0_volInd-2.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_PA_before_b0_volInd-3.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_PA_before_b0_volInd-4.nii.gz /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_PA_before_b0_volIn

0

#### Generate the acqparams.txt files

In [106]:
print('> Create the acqparams.txt files (one per session [''before'' and ''after''])')
# first get the totalReadoutTime from the json files
total_readout_time = {}
for scan in scansBaseNames.keys():
    with open(scansBaseNames[scan] + '.json') as json_file:        
        scanData = json.load(json_file)
        total_readout_time[scan] = scanData['TotalReadoutTime']

for time in ['before', 'after']:
    acqPars=[f'0 -1 0 {total_readout_time[f"AP_{time}"]}' for scan in sub_B0s_files if f'AP_{time}' in scan] + [f'0 1 0 {total_readout_time[f"PA_{time}"]}' for scan in sub_B0s_files if f'PA_{time}' in scan]
    # write a list of strings to a file (one string per line):
    with open(os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt'), 'w') as f:
        for item in acqPars:
            f.write("%s\n" % item)

> Create the acqparams.txt files (one per session [before and after])


#### Run TOPUP [currently printing the command]

In [126]:
for time in ['before', 'after']:
      print(f"qsub -pe smp 4 topup --imain={os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s \
            --datain={os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt')} \
            --config=b02b0.cnf \
            --out={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \
            --iout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout \
            --fout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout \
      ")

qsub -pe smp 4 topup --imain=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_PA_before_b0s             --datain=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_before_acqparams.txt             --config=b02b0.cnf             --out=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s             --iout=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout             --fout=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_fout       
qsub -pe smp 4 topup --imain=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_AP_PA_after_b0s             --datain=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_after_acqparams.txt             --config=b02b0.cnf             --out=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_after_b0s             --iout=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_after_b0s_iout             --

### (2) EDDY
* A tool for correcting Eddy currents (and motion)

#### Average TOPUP corrected (unwarped) B0 volums (the .iout file) [for now it prints the command]

In [19]:
for time in ['before', 'after']:
      print(f"fslmaths {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout -Tmean {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg")

fslmaths /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout -Tmean /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout_avg
fslmaths /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_after_b0s_iout -Tmean /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_after_b0s_iout_avg


#### Run BET on the averaged B0s volume [for now it prints the command]

In [20]:
for time in ['before', 'after']:
      print(f"bet {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain -m -f 0.2")

bet /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout_avg /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout_avg_brain -m -f 0.2
bet /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_after_b0s_iout_avg /Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_after_b0s_iout_avg_brain -m -f 0.2


#### Create index.txt file
This file maps the volumes in the main DTI data to the relevant line in the acqparams.txt and in the movpar.txt (assessed movement parameters from TOPUP) files
* details in: https://www.youtube.com/watch?v=1T1cRnX7MpA

In [37]:
# create an index.txt file for the EDDY
for time in ['before', 'after']:
    ind_to_write = 1
    with open(os.path.join(preproc_path, subjFolder, f'{time}_index.txt'), 'w') as f:
        for i in range(expectedVolums['AP']):   
            if i > 0 and i in expectedB0s_indxs[f'AP_{time}']:
                ind_to_write += 1
                print(ind_to_write)
            f.write("%s\n" %ind_to_write)


2
3
4
5
2
3
4
5


#### Run EDDY [currently printing the command]
* Correct for eddy currents and subject movement (and taking to account the suceptibility field calculated by TOPUP)

In [52]:
for time in ['before', 'after']:
     print(f"eddy --imain={scansBaseNames[f'AP_{time}']}.nii.gz \\\n\
          --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask \\\n\
          --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')}\\\n\
          --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} \\\n\
          --bvecs={scansBaseNames[f'AP_{time}']}.bvec \\\n\
          --bvals={scansBaseNames[f'AP_{time}']}.bval \\\n\
          --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
          --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} \\\n\
          --verbose \
     ")



eddy --imain=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz \
          --mask=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout_avg_brain_mask \
          --index=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/before_index.txt\
          --acqp=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_before_acqparams.txt \
          --bvecs=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.bvec \
          --bvals=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.bval \
          --topup=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s \
          --out=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/eddy_unwarped_images_sub-120_before \
          --verbose      
eddy --imain=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-02_dwi.nii.gz \
          --mask=/Users/ranigera/Dr

In [28]:
scansBaseNames[f'AP_{time}']

'/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-02_dwi'

In [27]:
scansBaseNames[f'AP_{time}']

{'AP_before': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi',
 'PA_before': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-pa_run-01_dwi',
 'AP_after': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-02_dwi',
 'PA_after': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-pa_run-02_dwi'}

In [ ]:
print('> Extract B0s (using the fslroi):')
sub_B0s_files = []
for scan in scansBaseNames.keys():
    for B0ind in B0s_indxs[scan]:
        B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
        print(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
        os.system(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
        sub_B0s_files.append(B0_file_name)


In [25]:
for scan in scansBaseNames.keys():
    print(scan)

AP_before
PA_before
AP_after
PA_after


In [18]:
 expectedB0s_indxs[f'AP_{time}']

[0, 1, 18, 35, 52]

In [31]:
scansBaseNames

{'AP_before': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi',
 'PA_before': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-pa_run-01_dwi',
 'AP_after': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-02_dwi',
 'PA_after': '/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-pa_run-02_dwi'}

In [22]:
os.system('ls')
subjFolder

'sub-120'

In [ ]:
os.system('sub-01_ses-1_acq-ap_run-01_dwi.bval')


In [227]:
B0s_indxs == expectedB0s_indxs

True

In [215]:
        raise Exception(f'The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')


Exception: The number of data points in the bval for one of the scans for subjetc 120 is not as expected.

In [206]:
pd.read_csv(scansBaseNames['AP_before'] + '.bval', header=None, sep=' ').T.shape[0]

69

In [ ]:
expectedVolums = {
    'AP': 69,
    'PA' : 7,
    }
expectedB0s = {


In [45]:
print(scanBaseName_AP_before), print(scanBaseName_PA_before)
print(scanBaseName_AP_after), print(scanBaseName_PA_after)

../DTI_tests/sub-110/ses-1/dwi/sub-110_ses-1_acq-ap_run-01_dwi
../DTI_tests/sub-110/ses-1/dwi/sub-110_ses-1_acq-pa_run-01_dwi
../DTI_tests/sub-110/ses-1/dwi/sub-110_ses-1_acq-ap_run-02_dwi
../DTI_tests/sub-110/ses-1/dwi/sub-110_ses-1_acq-pa_run-02_dwi


(None, None)

In [60]:
f'ses-{last_sess}/dwi/'

'ses-3/dwi/'

In [54]:
scanBaseName_AP_before  = f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-ap_run-01_dwi")}'
scanBaseName_PA_before  = f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-pa_run-01_dwi")}'

'../DTI_tests/sub-240/ses-1/dwi/sub-240_ses-1_acq-pa_run-01_dwi.nii.gz'

In [44]:
os.listdir(DWI_path)

NameError: name 'DWI_path' is not defined

In [37]:
os.listdir(data_path)

['sub-240', '.DS_Store', 'sub-110']

In [36]:
group == '3day'

True

In [28]:
int(299)>300

False